In [ ]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
import math
import random

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

<h1>Multiple Regression</h1>

Doing a simple linear regression is cool, but for this to be really useful we want to be able to use a bunch of factors to make a prediction. The process is largely the same, we just have more variables on the input (X) side. It is also difficult/impossible to visualize, since we'd need to see in 3+ dimensions. 

First, we'll get some data, and look at what the target and the features are going to be. This is a really simplifed example of some EDA that we normally do in advance of machine learning. 

In [ ]:
#Get data
#I'll drop density to make it more realistic.
df = pd.read_csv("bodyfat.csv")
df = df.drop(columns={"Density"})
df.head()

Cool. We'll predict the BodyFat (which is hard to accurately measure) by using the other stuff, which is easier to measure. 

Before we start, we want to do some exploration of the data.

In [ ]:
#Start Exploring the Data
df.describe()

At a glance, it looks like there aren't a lot of issues. One thing I notice is that there's a BodyFat reading of 0 for the min. That isn't possible, so we'll need to get rid of that. 

In [ ]:
#Get variable info. 
df.info()

All of our variables are correctly identified as numbers. Often if there's some erroneous data in a column it might get miscategorized as an object. Since we'll be dealing with non-numeric data in ML, starting next workbook, we probably want to double check.  

In [ ]:
#Visual correlation and distributions
sns.pairplot(df)

It appears there are some outliers that probably aren't helping - e.g. Height. 

In [ ]:
#Are there nulls?

##COMPLETE

In [ ]:
#Remove some outliers
#These were judgement cals. 
df = df[df["BodyFat"] > 5]
df = df[df["Height"] > 40]
df = df[df["Weight"] < 300]
df = df[df["Ankle"] < 30]
df.describe()

In [8]:
#Check with cleaned data. 
sns.pairplot(df)

Looks pretty good. We may want to examine some of the outliers a bit more closely, but overall the data looks pretty usable. Distributions are all pretty normal. Large outliers are gone. Nothing really jumps out as being a problem.

There does appear to be a large amount of correlation between variables. We'll worry about that later on, for now, looks good. Time to start regressing...
<br><br><br>
<h2>Setup the data</h2>

The basic idea here is the same as one variable regression, only instead of one X, we have a bunch. We can't do this with simple functions (thinkstats), at least not easily, so we'll use a package. 

<h3>SciKitLearn Data Setup</h3>

<ul>
<li>Y (target) = BodyFat. Goal shape - 1 column, one row for each row of data. 
<li>X (features) = All other columns of data. 1 column for each column of data, same number of rows as the target. 
</ul>

In [ ]:
#Y data is simple, do that first

##COMPLETE

Y is identical to the single variable stuff. For X, we have a width to our array. We can take a look again to see what we expect. 

In [ ]:
#Get a new df with only the features we'll use
#We'll use the DF later, so we want to make a separate DF, then make an array next step

##COMPLETE

In [ ]:
#Make that df into an array. 

##COMPLETE

In [ ]:
#shapes
print("X shape", x.shape)
print("Y shape", y.shape)

<h3>SKL Regress...</h3>

Shape for this one looks good! We have 240 rows, which matches the Y. We have 13 columns, which is what we get if we were to count up the columns by hand. Success!!

Time for regression stuff. 

In [ ]:
#Setup
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection

In [ ]:
#Generate model 

##COMPLETE

In [ ]:
#Get some info on our new regression model
r_sq = model.score(x, y)
print('R-squared:', r_sq)
print('Intercept:', model.intercept_[0])

#Our coefficent/slope is now an array of values - one per X. 
#Visualizing the regression would be a 14D space, where these are the slopes in each dimension. 
print('Coefs:', model.coef_[0])

<h3>Error Metrics</h3>

Get the residuals to calculate RMSE. 

The residual stuff is the same no matter how many Xs we have on the input side, since all we are checking is the Y values. 

In [ ]:
#Get RMSE

##COMPLETE

In [ ]:
#Get Residuals and picture them in a DF for easy reading. 
tmp1 = pd.DataFrame(y, columns={"Y values"})
tmp2 = pd.DataFrame(tmp, columns={"Predictions"})
tmp3 = pd.DataFrame((y-tmp), columns={"Residual"})
resFrame = pd.concat([tmp1,tmp2,tmp3], axis=1)
resFrame.head()

Easy!!

<h3>Statsmodels style</h3>

In [ ]:
#Statsmodels.
import statsmodels.api as sm

In [ ]:
#Fit the model. 

##COMPLETE

The x's are the input features in order. They are showing without their names because for the input we just used the arrays from the sklearn example above. Things like this are reasonably common when we're doing ML, as most other algorithms don't really give detailed data for each feature like linear regression does, so having them listed by names isn't super useful. To get the labels we could:
<ul>
<li>Feed the statsmodel a formula, like we have in the book. 
<li>Use a dataframe as an input. (Coming up)
<li>Create a list of column names and match the up after the fact - reconstructing output that you want from the values you want.
</ul>

In [ ]:
#RMSE

##COMPLETE

Add some descriptiveness to our statsmodels summary...

In [ ]:
#Make it more descriptive by using dataframe data or a formula
#Fit the model. 
#df of Xs that we made above is df_ here
X2 = sm.add_constant(df_)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

<h2>Multicollinearity - Useful, but Less Critical</h2>

For fun, we'll look at the multicollinearity, because it is crazy high in this example. Look back at the pairplot, most varaibles appear highly correlated with each other - as one increases, the other does as well. This makes sense logically, as these are all measures of the body size; as your wrist size increases, so does your forearm, and your bicep, etc... They are closely correlated. One effect of this is that the impact of each variable is hard to pinpoint, as all the varaibles are largely measuring the same thing. 

Addressing this won't make our accuracy measures get way better, but it will allow us to better attribute the impact to the individual features, which is one advantage to a linear regression - that table of results give us some data that we can use to edit our model. 

We can calculate the VIF - variance inflation factor. This is a measure of multicollinearity. The calculation is VIF = 1/(1-R2), so as R2 gets closer to 1, the VIF gets larger. If we think of it in R2 terms - a big R2 indicates that a large percentage in the varaince of the results is captured in the model. Here, the R2 is how much of the variance of the other features is explained by each feature. So if they are higly correlated that value will be high and R2 will be high, leading to a big VIF value. If the R2 is low, then the varaible doesn't capture varaiance in the other features, so it is different from them, or it captures different data. In this case the R2 is small, so the VIF is small. 

The rule of thumb is that a VIF of ~10 should be looked at. 

In [ ]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Function will check the VIF of each variable in a DF and return the results in another DF
def calc_vif(X):
    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [ ]:
#Check VIFs
calc_vif(df_)

All the scores are huge. We expected this since they all almost overlap on that pairplot earlier. What this tells us is that the information in each varaible has lots of overlap - in this case they all measure bodyfat, fairly directly. If we think about the real world impact of this it makese sense - the cheap and easy way to measure bodfat is by taking measurements and caliper pinches all over one's body. We'd expect the measurements to all be pretty correlated - it'd be a bit odd to have a huge wrist and a tiny ankle, or a big thigh and tiny knee. A few people might stand out, particularly very muscular people, but most people will be pretty consistent between every measurement based on fat level. 

How accurate will this model be if we drop a bunch of the high VIF stuff?

In [ ]:
#Remove some high VIF columns
df_2 = df_.drop(columns={"Hip", "Knee", "Wrist", "Chest","Neck","Thigh","Ankle"})

In [ ]:
#Run statsmodesl to get results
X2 = sm.add_constant(df_2)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#Recheck
calc_vif(df_2)

In [ ]:
#RMSE for smaller model 
x2 = np.array(df_2)
x2.shape
model2 = LinearRegression().fit(x2,y)
ypred2 = model2.predict(x2)
mean_squared_error(y,ypred2,squared=False)

<h3>Smaller Model Results</h3>

Removing variables doesn't up the R2 - we'll explore that more later on in ML.

The RMSE doesn't get much worse (and the change is within the expectation of randomness here) - that makes sense because what we found with this multicollinearity was that each varaible contained the same information. So when we remove one duplicate piece of that information, not much changes. This process is called feature selection, it is one of the big things in ML, and we'll look at it in different ways as we do next semester. 

When we are just using CSVs full of data, there isn't a very large impact in removing a few variables like this. If we were dealing with data that we had to collect - e.g. if we were paying nurses to do all of these measurements on people - then we'd want to minimize the number of features needed, so our nurses could be faster and therefore cheaper. Similarly, if we had massive amounts of data and training our model or storing the data was costly, being able to be accurate with less data is good. We don't often run into constraints in learning, because the data is small, but they can exist. 

In [ ]:
#We can cut it down to the bare-bones - I'll take the vars with small p values
df_3 = df[["Abdomen","Weight"]]
X2 = sm.add_constant(df_3)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#RMSE for tiny model 
x2 = np.array(df_3)
x2.shape
model2 = LinearRegression().fit(x2,y)
ypred2 = model2.predict(x2)
mean_squared_error(y,ypred2,squared=False)

What's the moral of the story? We can predict almost as well with just two varaibles as we did with all of them!

Challenge - run this with a split of the data. One part to train, another to test. Not the accuracy differences...

In [ ]:
#Run above with test/train split